In [ ]:
import cv2

camera_img = cv2.imread("image1.png")
map_img = cv2.imread("image2.png")

gray1 = cv2.cvtColor(camera_img, cv2.COLOR_BGRA2GRAY)
gray2 = cv2.cvtColor(map_img, cv2.COLOR_BGRA2GRAY)


In [ ]:
orb = cv2.ORB_create()

kp1, des1 = orb.detectAndCompute(gray1, None)
kp2, des2 = orb.detectAndCompute(gray2, None)

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1, des2)

matches = sorted(matches, key=lambda x: x.distance)

In [ ]:
import matplotlib.pyplot as plot

img_matches = cv2.drawMatches(camera_img, kp1, map_img, kp2, matches[:50], None, 
                              flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

plot.imshow(img_matches)
plot.show()

In [ ]:
cv2.imwrite('matches.jpg', img_matches)

In [ ]:
import numpy as np
pt1 = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
pt2 = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

H, mask = cv2.findHomography(pt1, pt2, cv2.RANSAC, 5.0)

height, width, _ = map_img.shape
img_aligned = cv2.warpPerspective(camera_img, H, (width, height))

plot.imshow(img_aligned)
plot.show()